# 07_C_Experiment_1_8_2

## This is a secondary file to run and experiment around for Experiment 1.

Files 07_A is the primary file for the code. This file was used to speed up computation and allow parallel coding.


### Sources used 

We use the EvalHook and MyTrainer function explain in  https://eidos-ai.medium.com/training-on-detectron2-with-a-validation-set-and-plot-loss-on-it-to-avoid-overfitting-6449418fbf4e and documented in  https://gist.github.com/ortegatron/c0dad15e49c2b74de8bb09a5615d9f6b to allow calculation of the validation loss. 
We adapt it using a hook addtion taken from https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/287023 to allow for model saving. 

We have further done abstractions on https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5 provided by the Detectron2 Team. 

In [1]:
!pip install pyyaml==5.1
#!pip uninstall torch
!yes | pip uninstall detectron2
#!pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
    
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Found existing installation: detectron2 0.6+cu111
Uninstalling detectron2-0.6+cu111:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/detectron2-0.6+cu111.dist-info/*
    /usr/local/lib/python3.8/dist-packages/detectron2/*
    /usr/local/lib/python3.8/dist-packages/tools/*
Proceed (y/n)?   Successfully uninstalled detectron2-0.6+cu111
yes: standard output: Broken pipe
Looking in links: https://download.pytorch.org/whl/torch_stable.html
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
  Using cached https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/detectron2-0.6%2Bcu111-cp38-cp38-linux_x86_64.whl (6.9 MB)


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [18]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
torch:  1.9 ; cuda:  cu111
detectron2: 0.6


In [19]:
#!pip install datetime

In [19]:
import os
import sys

In [20]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [21]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data import build_detection_train_loader
from detectron2.data import DatasetMapper
from detectron2.data import DatasetCatalog

from detectron2.checkpoint import DetectionCheckpointer, PeriodicCheckpointer

from detectron2.config import get_cfg
from detectron2 import model_zoo

In [22]:
import json
from datetime import datetime

In [6]:
# Error Message while training: module 'distutils' has no attribute 'version'
#!python3 -m pip install setuptools==58.2.0
# then restart the kernel

In [23]:
## Function to register our datasets 
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog

#dataset_name_train = "thesis_030_train"
#dataset_name_val = "thesis_030_test"

def register(name, path_labels, path_data):

    # Check if dataset name already exists in catalog
    if name in DatasetCatalog.list():
        
        # Remove existing dataset from catalog
        DatasetCatalog.remove(name)
        MetadataCatalog.remove(name)

    # Register your dataset using register_coco_instances
    register_coco_instances(name, {}, path_labels, path_data)

# if it throws an error that name is already registered, just change the name... 
#register(dataset_name_train, "/workspace/data/labels_train.json", "/workspace/data/data_030/train/data")
#register(dataset_name_val, "/workspace/data/labels_test.json", "/workspace/data/data_030/val/data")

In [24]:
base_path = "/workspace/scripts"

os.chdir(base_path)

In [45]:
# Taken from https://eidos-ai.medium.com/training-on-detectron2-with-a-validation-set-and-plot-loss-on-it-to-avoid-overfitting-6449418fbf4e
# https://gist.github.com/ortegatron/c0dad15e49c2b74de8bb09a5615d9f6b

from EvalLossHook import LossEvalHook
from detectron2.engine import BestCheckpointer
from detectron2.checkpoint import DetectionCheckpointer

class MyTrainer(DefaultTrainer):
        
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)
                     
    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1,LossEvalHook(
            self.cfg.TEST.EVAL_PERIOD,
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg,True)
            )
        ))
        
        # add the best checkpointer hook
        # adapted from https://www.kaggle.com/c/sartorius-cell-instance-segmentation/discussion/287023
        hooks.insert(-1, BestCheckpointer(self.cfg.TEST.EVAL_PERIOD, 
                                         DetectionCheckpointer(self.model, self.cfg.OUTPUT_DIR),
                                         "bbox/AP50",
                                         "max",
                                         "best_ap50"
                                         ))
        return hooks

    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg)
 


In [9]:
# Lukas Univ Edinburgh Trainer Version 

class OurTrainer(DefaultTrainer):

    def __init__(self, cfg):
        super().__init__(cfg)

        if isinstance(cfg.DATASETS.EVAL, str):
            self.eval_datasets = [cfg.DATASETS.EVAL]
        else:
            self.eval_datasets = cfg.DATASETS.EVAL

        # prepare evaluation
        self.eval_loaders = []
        self.evaluators = []
        for dataset in self.eval_datasets:

            loader = build_detection_test_loader(DatasetCatalog.get(dataset), 
                                                 mapper=DatasetMapper(cfg, is_train=False))

            self.eval_loaders.append(loader)
            self.evaluators.append(COCOEvaluator(
                dataset,
                output_dir=cfg.OUTPUT_DIR 
                ))
    
    def build_train_loader(cls, cfg):
#         if cfg.SOLVER.STRONG_AUGMENT:
#             print("STRONG AUGMENTATIONS TO TRAINING SET")
#             return build_detection_train_loader(cfg, mapper=DatasetMapperAugment(cfg, is_train=True)) 
#         else:
#             return build_detection_train_loader(cfg)
        return build_detection_train_loader(cfg)

    
    def after_step(self):
        super().after_step()

        if (self.iter+1) % self.cfg.TEST.INTERVAL == 0:                                   
    
            for dataset, loader, evaluator in zip(self.eval_datasets, 
                                                    self.eval_loaders,
                                                    self.evaluators):
    
                results = inference_on_dataset(self.model,
                                                loader,
                                                evaluator)
                with open(
                    os.path.join(
                        self.cfg.OUTPUT_DIR,
                        'eval_'+dataset+'_iter_'+str(self.iter)+'.json'),
                        'w') as out:
                    json.dump(results, out)

In [10]:
# put it all in a function to be able to build a loop for the different resolutions
from detectron2.checkpoint import DetectionCheckpointer, PeriodicCheckpointer


def train_detectron_standard(dataset_name_train, 
                             dataset_name_val, 
                             resolution,
                             iterations,
                             eval_period = None,
                             batch_size = 8,
                             anchors = [[4, 8, 16, 32, 64]],
                             weight_decay = 0.0001,
                             solver_steps_divider = 10,
                             freeze_at = 2
                            ):
    
    """
    func: fine tunes a detectron model
    
    inputs:
        - dataset_name_train: name of the coco registered training dataset
        - dataset_name_val:   name of the coco registered validation dataset
        - resolution:         input resolution of the images
        - iterations:         number of iterations the model should train
        - eval_period:        (default = None) number of iterations after which the model is evaluated on the train set
        - batch_size:         (default = 8)
        - anchors:            (default = [[4, 8, 16, 32, 64]]) size of the ROI heads proposals 
        - weight_decay:       (default = 0.0001) regularization (float)
        - solver_steps_divider(default = 10) after what fraction of the number of iterations the LR is multiplied by 0.1
        
    returns: 
        - cfg object
        - trainer object
        - predictor object
    """
        
    now = datetime.now() 
    output_dir = f"/workspace/output/output_new_res{resolution}_{now.strftime('%m.%d.%H.%M')}"


    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
    cfg.INPUT.FORMAT = "RGB"
    
    cfg.OUTPUT_DIR = output_dir
    cfg.DATASETS.TRAIN = (dataset_name_train)
    cfg.DATASETS.TEST = (dataset_name_val,)
    cfg.DATASETS.EVAL = (dataset_name_val,)

    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.MODEL.ANCHOR_GENERATOR.SIZES = anchors
    cfg.SOLVER.MAX_ITER = iterations
    cfg.TEST.EVAL_PERIOD = eval_period # für unseren Evaluator

    cfg.SOLVER.IMS_PER_BATCH = batch_size
    cfg.SOLVER.WEIGHT_DECAY = weight_decay
    #cfg.SOLVER.STEPS = [i for i in range(0, iterations, round(iterations/solver_steps_divider))][1:]
    #print(cfg.SOLVER.STEPS)
    
    cfg.MODEL.BACKBONE.FREEZE_AT = freeze_at
    print("freeeeezeeeee:", cfg.MODEL.BACKBONE.FREEZE_AT)
    
    #cfg.SOLVER.CHECKPOINT_PERIOD = 
      
    
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    cfg.TEST.DETECTIONS_PER_IMAGE = 30

    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    
    trainer = MyTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()
    
    
    predictor = DefaultPredictor(cfg)
    
    return cfg, trainer, predictor

def evaluate_detectron(cfg, predictor_obj, resolution, dataset_name_val): 

    """
    func:   evaluates a trained detectron model

    input:
    cfg:                configuration object of the model
    predictor_obj:      the predictor object of the same model 
    resolution:         the resolution of the images
    dataset_name_val:   the COCO registered name of the val dataset

    returns:            dictionary with model metrics 
    """

    evaluator = COCOEvaluator(dataset_name_val, output_dir="/workspace/output/output"+str(resolution))
    val_loader = build_detection_test_loader(cfg, dataset_name_val)
    
    return inference_on_dataset(predictor_obj.model, val_loader, evaluator)

## Hyperparamter Tuning

In [12]:
def restructure_labels(labels_file, image_dir):
    with open(labels_file, 'r') as f:
        labels_data = json.load(f)

    file_data = []
    for img in labels_data['images']:
        file_name = img['file_name']
        img_id = img['id']
        height = img['height']
        width = img['width']
        img_path = image_dir + '/' + file_name

        annotations = []
        for anno in labels_data['annotations']:
            if anno['image_id'] == img_id:
                bbox = anno['bbox']
                bbox_mode = 0
                category_id = anno['category_id']
                annotations.append({
                    'bbox': bbox,
                    'bbox_mode': bbox_mode,
                    'category_id': category_id
                })

        file_data.append({
            'file_name': img_path,
            'height': height,
            'width': width,
            'image_id': img_id,
            'annotations': annotations
        })

        return file_data

In [13]:
#from detectron2.utils.comm import create_local_process_group # show the import here as reference 

def train_detectron_tune(config_dict = None, 
                         default_dict = None,
                         resolution = "res_missing",
                         output_dir_prefix = None,
                         trial_id = None,
                         dataset_name_train = '030_train',
                        dataset_name_val = '030_val'
                        ):
    
    """
    func: fine tunes a detectron model
    
    inputs:
        - dataset_name_train: name of the coco registered training dataset
        - dataset_name_val:   name of the coco registered validation dataset
        - resolution:         input resolution of the images
        - iterations:         number of iterations the model should train
        - eval_period:        (default = None) number of iterations after which the model is evaluated on the train set
        - batch_size:         (default = 8)
        - anchors:            (default = [[4, 8, 16, 32, 64]]) size of the ROI heads proposals 
        - weight_decay:       (default = 0.0001) regularization (float)
        - solver_steps_divider(default = 10) after what fraction of the number of iterations the LR is multiplied by 0.1
        
    returns: 
        - cfg object
        - trainer object
        - predictor object
    """

    if config_dict is None:
        config_dict = {}
    if default_dict is None:
        default_dict = {
            'eval_period': None,
            'learning_rate': 0.001,
            'batch_size': 8,
            'anchors': [[4, 8, 16, 32, 64]],
            'weight_decay': 0.0001,
            'solver_steps_divider': 10,
            'freeze_at': 2,
            'gradient_clipping': False}

    # Merge default_dict with my_dict, overriding any duplicate keys
    config = {**default_dict, **config_dict}
    
    print(config)
    
    register(dataset_name_train, "/workspace/data/labels_train_DT.json", f"/workspace/data/data_{resolution}/train/data")
    register(dataset_name_val, "/workspace/data/labels_test_DT.json", f"/workspace/data/data_{resolution}/test/data")
    
    
    if output_dir_prefix:
        now = datetime.now() 
        output_dir = f"/workspace/output/{output_dir_prefix}/res{resolution}_{trial_id}"
    else:
        now = datetime.now() 
        output_dir = f"/workspace/output/res{resolution}_{now.strftime('%m.%d.%H.%M')}"
        
    #create_local_process_group(num_workers_per_machine=2)
    
    # batchsize * iterations
    views = 120000
    
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
    cfg.INPUT.FORMAT = "RGB"
    
    cfg.OUTPUT_DIR = output_dir
    print(cfg.OUTPUT_DIR)
    cfg.DATASETS.TRAIN = (dataset_name_train)
    cfg.DATASETS.TEST = (dataset_name_val,)
    cfg.DATASETS.EVAL = (dataset_name_val,)

    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.MODEL.ANCHOR_GENERATOR.SIZES = config['anchors']
    cfg.SOLVER.MAX_ITER = round(views/config['batch_size'])
    cfg.TEST.EVAL_PERIOD = cfg.SOLVER.MAX_ITER / 15 # für unseren Evaluator

    cfg.SOLVER.IMS_PER_BATCH = config['batch_size']
    cfg.SOLVER.BASE_LR = config['learning_rate']
    cfg.SOLVER.WEIGHT_DECAY = config['weight_decay']
    cfg.SOLVER.CLIP_GRADIENTS.ENABLED = config['gradient_clipping']
    cfg.SOLVER.STEPS = [.5*cfg.SOLVER.MAX_ITER, .75*cfg.SOLVER.MAX_ITER, .9*cfg.SOLVER.MAX_ITER]
    #print(cfg.SOLVER.STEPS)
    
    cfg.MODEL.BACKBONE.FREEZE_AT = config['freeze_at']
      
    
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    cfg.TEST.DETECTIONS_PER_IMAGE = 30

    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    
    trainer = MyTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()
    
    
    predictor = DefaultPredictor(cfg)
    
    return cfg, trainer, predictor

In [16]:
import random as rd
import itertools


def get_combinations(d):
    keys = d.keys()
    values = d.values()
    combinations = list(itertools.product(*values))
    return [dict(zip(keys, c)) for c in combinations]

def load_json_arr(json_path):
    lines = []
    with open(json_path, 'r') as f:
        for line in f:
            lines.append(json.loads(line))
    return lines

def tune_bruteforce(search_space, max_trials = None):
    
    now = datetime.now() 
    unravelled_space = get_combinations(search_space)
    output_dir_prefix = f"hyper_tune_8_{now.strftime('%m.%d.%H.%M')}"
    
    # if there is no specification run a full grid search
    if max_trials != None:
        unravelled_space = rd.sample(unravelled_space, max_trials)
        
        
    print(f"Saving to: {output_dir_prefix}")
    print(f"Beginning to search in: {unravelled_space}")
    
    trial_dict = {}
    
    if not os.path.exists(f'/workspace/output/{output_dir_prefix}'):
        os.mkdir(f'/workspace/output/{output_dir_prefix}')
    
    ap50 = 0
    best_config = None
    
    
    for t, config in enumerate(unravelled_space):
        
        trial = {'config': config}
        
        try:
            cfg, trainer, predictor = train_detectron_tune(config, 
                             resolution = '030',
                            output_dir_prefix = output_dir_prefix,
                            dataset_name_train = '030_train',
                            dataset_name_val = '030_val',
                            trial_id = t)
        
        
            trial['metrics_path'] = cfg.OUTPUT_DIR
        
            metrics = load_json_arr(cfg.OUTPUT_DIR + '/metrics.json')
            trial['maxAP50'] = max([met["bbox/AP50"] for met in metrics if 'bbox/AP50' in met and 'validation_loss' in met])

            if trial['maxAP50'] > ap50:
                ap50 = trial['maxAP50']
                best_config = t
        except:
            trial['failed'] = True
        
        trial_dict[t] = trial 
        
    with open(f'/workspace/output/{output_dir_prefix}/results_tuning.json', 'w') as f:
        json.dump(trial_dict, f)
    
    return ap50, best_config, trial_dict
        

In [19]:
# output_tuning = tune_bruteforce(search_space = {
#         "learning_rate": [1/(10**i) for i in range(2,5)],
#         "batch_size": [8],
#         'gradient_clipping': [True, False],
#         'weight_decay': [1/(10**i) for i in range(3,5)],
#         'freeze_at': [1,2]
#     })

## Train on different resolutions 

In [14]:
#os.environ["CUDA_VISIBLE_DEVICES"] = "2"
torch.cuda.empty_cache()

In [17]:
# train the model for all our resolutions on a given config setting 

best_config = {'learning_rate': 0.001, 
               'batch_size': 8, 'gradient_clipping': False, 'weight_decay': 0.001, 'freeze_at': 2}


resolution_set = ['035','040','045','050','070','100']

now = datetime.now() 
output_dir_prefix = f"experiment1_test__conf3_{now.strftime('%m.%d.%H.%M')}"
if not os.path.exists(f'/workspace/output/{output_dir_prefix}'):
    os.mkdir(f'/workspace/output/{output_dir_prefix}')
    

ap50 = 0
highest_config = None
trial_dict = {}

print(f"Saving to: {output_dir_prefix}")



for res in resolution_set:

    # define the names 
    dataset_name_train = "tower_train_" + str(res)
    dataset_name_val = "tower_val_" + str(res)
    

    trial = {'config': best_config}


    cfg, trainer, predictor = train_detectron_tune(best_config, 
                     resolution = res,
                    output_dir_prefix = output_dir_prefix,
                    dataset_name_train = dataset_name_train,
                    dataset_name_val = dataset_name_val,
                    trial_id = int(res))


    trial['metrics_path'] = cfg.OUTPUT_DIR

    metrics = load_json_arr(cfg.OUTPUT_DIR + '/metrics.json')
    trial['maxAP50'] = max([met["bbox/AP50"] for met in metrics if 'bbox/AP50' in met and 'validation_loss' in met])

    if trial['maxAP50'] > ap50:
        ap50 = trial['maxAP50']
        highest_config = int(res)

    trial['failed'] = True
        
    trial_dict[res] = trial 
        
with open(f'/workspace/output/{output_dir_prefix}/results_tuning.json', 'w') as f:
    json.dump(trial_dict, f)
print(ap50, highest_config)

Saving to: experiment1_test__conf3_04.13.10.25
{'eval_period': None, 'learning_rate': 0.001, 'batch_size': 8, 'anchors': [[4, 8, 16, 32, 64]], 'weight_decay': 0.001, 'solver_steps_divider': 10, 'freeze_at': 2, 'gradient_clipping': False}
/workspace/output/experiment1_test__conf3_04.13.10.25/res035_35
[04/13 10:25:57 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 

WARNING [04/13 10:25:57 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/13 10:25:57 d2.data.datasets.coco]: Loaded 11696 images in COCO format from /workspace/data/labels_train_DT.json
[04/13 10:25:57 d2.data.build]: Removed 848 images with no usable annotations. 10848 images left.
[04/13 10:25:58 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[04/13 10:25:58 d2.data.build]: Using training sampler TrainingSampler
[04/13 10:25:58 d2.data.common]: Serializing 10848 elements to byte tensors and concatenating them all ...
[04/13 10:25:58 d2.data.common]: Serialized dataset takes 5.53 MiB
[04/13 10:25:58 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='ch

[04/13 10:25:58 d2.checkpoint.c2_model_loading]: Renaming Caffe2 weights ......
[04/13 10:25:58 d2.checkpoint.c2_model_loading]: Following weights matched with submodule backbone.bottom_up:
| Names in Model    | Names in Checkpoint       | Shapes                                          |
|:------------------|:--------------------------|:------------------------------------------------|
| res2.0.conv1.*    | res2_0_branch2a_{bn_*,w}  | (64,) (64,) (64,) (64,) (64,64,1,1)             |
| res2.0.conv2.*    | res2_0_branch2b_{bn_*,w}  | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| res2.0.conv3.*    | res2_0_branch2c_{bn_*,w}  | (256,) (256,) (256,) (256,) (256,64,1,1)        |
| res2.0.shortcut.* | res2_0_branch1_{bn_*,w}   | (256,) (256,) (256,) (256,) (256,64,1,1)        |
| res2.1.conv1.*    | res2_1_branch2a_{bn_*,w}  | (64,) (64,) (64,) (64,) (64,256,1,1)            |
| res2.1.conv2.*    | res2_1_branch2b_{bn_*,w}  | (64,) (64,) (64,) (64,) (64,64,3,3)             |
| res2.1.c

Some model parameters or buffers are not found in the checkpoint:
backbone.fpn_lateral2.{bias, weight}
backbone.fpn_lateral3.{bias, weight}
backbone.fpn_lateral4.{bias, weight}
backbone.fpn_lateral5.{bias, weight}
backbone.fpn_output2.{bias, weight}
backbone.fpn_output3.{bias, weight}
backbone.fpn_output4.{bias, weight}
backbone.fpn_output5.{bias, weight}
proposal_generator.rpn_head.anchor_deltas.{bias, weight}
proposal_generator.rpn_head.conv.{bias, weight}
proposal_generator.rpn_head.objectness_logits.{bias, weight}
roi_heads.box_head.fc1.{bias, weight}
roi_heads.box_head.fc2.{bias, weight}
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
The checkpoint state_dict contains keys that are not used by the model:
  fc1000.{bias, weight}


[04/13 10:25:58 d2.engine.train_loop]: Starting training from iteration 0
[04/13 10:26:06 d2.utils.events]:  eta: 1:43:16  iter: 19  total_loss: 1.199  loss_cls: 0.4266  loss_box_reg: 0.001305  loss_rpn_cls: 0.6994  loss_rpn_loc: 0.06806  time: 0.4006  data_time: 0.0639  lr: 1.9981e-05  max_mem: 8529M
[04/13 10:26:15 d2.utils.events]:  eta: 1:44:16  iter: 39  total_loss: 0.806  loss_cls: 0.0487  loss_box_reg: 0.001725  loss_rpn_cls: 0.6931  loss_rpn_loc: 0.06455  time: 0.4089  data_time: 0.0577  lr: 3.9961e-05  max_mem: 8529M
[04/13 10:26:23 d2.utils.events]:  eta: 1:44:24  iter: 59  total_loss: 0.8018  loss_cls: 0.05355  loss_box_reg: 0.002454  loss_rpn_cls: 0.6771  loss_rpn_loc: 0.06141  time: 0.4101  data_time: 0.0589  lr: 5.9941e-05  max_mem: 8529M
[04/13 10:26:31 d2.utils.events]:  eta: 1:44:41  iter: 79  total_loss: 0.7716  loss_cls: 0.04926  loss_box_reg: 0.001453  loss_rpn_cls: 0.6534  loss_rpn_loc: 0.05794  time: 0.4126  data_time: 0.0579  lr: 7.9921e-05  max_mem: 8529M
[04/13

...

[04/13 20:54:39 d2.utils.events]:  eta: 0:11:30  iter: 13379  total_loss: 0.6475  loss_cls: 0.2354  loss_box_reg: 0.355  loss_rpn_cls: 0.02223  loss_rpn_loc: 0.02773  validation_loss: 0.8524  time: 0.4209  data_time: 0.0588  lr: 1e-05  max_mem: 8532M
[04/13 20:54:47 d2.utils.events]:  eta: 0:11:22  iter: 13399  total_loss: 0.6347  loss_cls: 0.2299  loss_box_reg: 0.347  loss_rpn_cls: 0.02135  loss_rpn_loc: 0.02805  validation_loss: 0.8524  time: 0.4209  data_time: 0.0551  lr: 1e-05  max_mem: 8532M
[04/13 20:54:56 d2.utils.events]:  eta: 0:11:13  iter: 13419  total_loss: 0.6494  loss_cls: 0.2185  loss_box_reg: 0.3693  loss_rpn_cls: 0.02613  loss_rpn_loc: 0.03238  validation_loss: 0.8524  time: 0.4210  data_time: 0.0568  lr: 1e-05  max_mem: 8532M
[04/13 20:55:04 d2.utils.events]:  eta: 0:11:05  iter: 13439  total_loss: 0.6504  loss_cls: 0.2379  loss_box_reg: 0.3629  loss_rpn_cls: 0.01987  loss_rpn_loc: 0.03043  validation_loss: 0.8524  time: 0.4210  data_time: 0.0576  lr: 1e-05  max_mem: 

[04/13 20:59:01 d2.data.common]: Serializing 2701 elements to byte tensors and concatenating them all ...
[04/13 20:59:01 d2.data.common]: Serialized dataset takes 1.22 MiB
WARNING [04/13 20:59:01 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[04/13 20:59:01 d2.evaluation.evaluator]: Start inference on 2701 batches
[04/13 20:59:02 d2.evaluation.evaluator]: Inference done 11/2701. Dataloading: 0.0007 s/iter. Inference: 0.0351 s/iter. Eval: 0.0002 s/iter. Total: 0.0359 s/iter. ETA=0:01:36
[04/13 20:59:07 d2.evaluation.evaluator]: Inference done 150/2701. Dataloading: 0.0009 s/iter. Inference: 0.0350 s/iter. Eval: 0.0002 s/iter. Total: 0.0361 s/iter. ETA=0:01:32
[04/13 20:59:12 d2.evaluation.evaluator]: Inference done 289/2701. Dataloading: 0.0009 s/iter. Inference: 0.0350 s/iter. Eval: 0.0001 s/iter. Total: 0.0361 s/iter. ETA=0:01:27
[04/13 20:59:17 d2.evaluation.evaluator]: Inference done

[04/13 21:03:46 d2.utils.events]:  eta: 0:06:07  iter: 14139  total_loss: 0.7011  loss_cls: 0.236  loss_box_reg: 0.3869  loss_rpn_cls: 0.02334  loss_rpn_loc: 0.03394  validation_loss: 0.8542  time: 0.4210  data_time: 0.0539  lr: 1e-06  max_mem: 8532M
[04/13 21:03:54 d2.utils.events]:  eta: 0:05:58  iter: 14159  total_loss: 0.6344  loss_cls: 0.2228  loss_box_reg: 0.3571  loss_rpn_cls: 0.02234  loss_rpn_loc: 0.02972  validation_loss: 0.8542  time: 0.4210  data_time: 0.0529  lr: 1e-06  max_mem: 8532M
[04/13 21:04:03 d2.utils.events]:  eta: 0:05:50  iter: 14179  total_loss: 0.6753  loss_cls: 0.238  loss_box_reg: 0.3549  loss_rpn_cls: 0.02541  loss_rpn_loc: 0.03019  validation_loss: 0.8542  time: 0.4210  data_time: 0.0552  lr: 1e-06  max_mem: 8532M
[04/13 21:04:11 d2.utils.events]:  eta: 0:05:41  iter: 14199  total_loss: 0.6405  loss_cls: 0.2339  loss_box_reg: 0.3561  loss_rpn_cls: 0.02182  loss_rpn_loc: 0.02929  validation_loss: 0.8542  time: 0.4210  data_time: 0.0560  lr: 1e-06  max_mem: 

[04/13 21:08:24 d2.utils.events]:  eta: 0:01:25  iter: 14799  total_loss: 0.6069  loss_cls: 0.2331  loss_box_reg: 0.3414  loss_rpn_cls: 0.02611  loss_rpn_loc: 0.02871  validation_loss: 0.8542  time: 0.4210  data_time: 0.0575  lr: 1e-06  max_mem: 8532M
[04/13 21:08:32 d2.utils.events]:  eta: 0:01:16  iter: 14819  total_loss: 0.6384  loss_cls: 0.2288  loss_box_reg: 0.371  loss_rpn_cls: 0.01851  loss_rpn_loc: 0.02656  validation_loss: 0.8542  time: 0.4209  data_time: 0.0545  lr: 1e-06  max_mem: 8532M
[04/13 21:08:40 d2.utils.events]:  eta: 0:01:08  iter: 14839  total_loss: 0.6818  loss_cls: 0.2308  loss_box_reg: 0.3849  loss_rpn_cls: 0.02332  loss_rpn_loc: 0.02915  validation_loss: 0.8542  time: 0.4209  data_time: 0.0549  lr: 1e-06  max_mem: 8532M
[04/13 21:08:49 d2.utils.events]:  eta: 0:00:59  iter: 14859  total_loss: 0.6569  loss_cls: 0.2316  loss_box_reg: 0.3639  loss_rpn_cls: 0.02049  loss_rpn_loc: 0.02679  validation_loss: 0.8542  time: 0.4209  data_time: 0.0564  lr: 1e-06  max_mem:

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## visualize predictions

In [ ]:
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import ColorMode

import random
import cv2
import matplotlib.pyplot as plt

def plot_sample(dataset_name, n=1):
    
    """
    Plots a random sample of images and annotations from a Detectron2 dataset.

    Args:
    - dataset_name (str): the name of the dataset registered with Detectron2.
    - n (int): the number of images to plot. Default is 1.

    Returns:
    - None
    """
    
    dataset_custom = DatasetCatalog.get(dataset_name)
    dataset_custom_metadata = MetadataCatalog.get(dataset_name)
    image_filepaths = random.sample(dataset_custom, n)
    for s in image_filepaths:
        
        img = cv2.imread(s["file_name"])
        v = Visualizer(img[:, :, ::-1], metadata=dataset_custom_metadata, scale=0.5)
        v = v.draw_dataset_dict(s)
        plt.figure(figsize=(12,12))
        plt.imshow(v.get_image())
        return plt.show()
    
    
def plot_prediction(dataset_name, predictor, n=1):
    
    """
    Plots a random image from a Detectron2 dataset and its corresponding predictions generated by the provided predictor.

    Args:
    - dataset_name (str): the name of the dataset registered with Detectron2.
    - predictor: a Detectron2 predictor object that takes an image as input and generates predictions.
    - n (int): the number of images to plot. Default is 1.

    Returns:
    - None
    """
    
    dataset_custom = DatasetCatalog.get(dataset_name)
    dataset_custom_metadata = MetadataCatalog.get(dataset_name)
    image_filepaths = random.sample(dataset_custom, n)
    print(len(image_filepaths))
    
    for s in image_filepaths: 
        im = cv2.imread(s["file_name"])
        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(24, 12))
        
        
        v = Visualizer(im[:, :, ::-1], metadata=dataset_custom_metadata, scale=0.5)
        v = v.draw_dataset_dict(s)
        ax[0].imshow(v.get_image())
        ax[0].axis('off')
        ax[0].set_title('Original Image')     
        
        
        outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
        v = Visualizer(im,#[:, :, ::-1],
                       metadata= MetadataCatalog.get(dataset_name), 
                       scale=0.5 
                       #,instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
        )
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        
        ax[1].imshow(out.get_image()[:, :, ::-1])
        ax[1].axis('off')
        ax[1].set_title('Predicted Image')

    return plt.show()
    

In [ ]:
plot_prediction(dataset_name_train, predictor_obj, 5)

## Evaluate best model on test set 

dazu: register function
dazu: MyTrainer


In [68]:
dataset_name_train = "tower_train_030"
dataset_name_val = "tower_val_030"


register(dataset_name_train, "/workspace/data/labels_train_DT.json", f"/workspace/data/data_030/train/data")
register(dataset_name_val, "/workspace/data/labels_test_DT.json", f"/workspace/data/data_030/test/data")
    

In [69]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.TEST.DETECTIONS_PER_IMAGE = 30
cfg.MODEL.WEIGHTS = "/workspace/output/hyper_tune_5_04.05.14.09/res030_8/best_ap50.pth"

cfg.DATASETS.TRAIN = (dataset_name_train)
cfg.DATASETS.TEST = (dataset_name_val,)
cfg.DATASETS.EVAL = (dataset_name_val,)

cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[4,8,16,32,64]]
cfg.SOLVER.MAX_ITER = 24000
cfg.TEST.EVAL_PERIOD = cfg.SOLVER.MAX_ITER / 15 # 

cfg.SOLVER.IMS_PER_BATCH = 5
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.WEIGHT_DECAY = 0.001
cfg.SOLVER.CLIP_GRADIENTS.ENABLED = True
cfg.SOLVER.STEPS = [.5*cfg.SOLVER.MAX_ITER, .75*cfg.SOLVER.MAX_ITER, .9*cfg.SOLVER.MAX_ITER]
#print(cfg.SOLVER.STEPS)

cfg.MODEL.BACKBONE.FREEZE_AT = 1


cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.TEST.DETECTIONS_PER_IMAGE = 30


trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=True)

predictor = DefaultPredictor(cfg)

[04/14 12:20:35 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

WARNING [04/14 12:20:35 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/14 12:20:35 d2.data.datasets.coco]: Loaded 11696 images in COCO format from /workspace/data/labels_train_DT.json
[04/14 12:20:35 d2.data.build]: Removed 848 images with no usable annotations. 10848 images left.
[04/14 12:20:36 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[04/14 12:20:36 d2.data.build]: Using training sampler TrainingSampler
[04/14 12:20:36 d2.data.common]: Serializing 10848 elements to byte tensors and concatenating them all ...
[04/14 12:20:36 d2.data.common]: Serialized dataset takes 5.53 MiB
[04/14 12:20:36 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='ch

In [70]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("tower_val_030", cfg, False, output_dir="/workspace/final_test",max_dets_per_image = 30)
val_loader = build_detection_test_loader(cfg, "tower_val_030")

#val_loader = build_detection_test_loader(DatasetCatalog.get("tower_val_030"), 
                                                 #mapper=DatasetMapper(cfg, is_train=False))

print(inference_on_dataset(trainer.model, val_loader, evaluator))

WARNING [04/14 12:20:38 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
WARNING [04/14 12:20:38 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/14 12:20:38 d2.data.datasets.coco]: Loaded 2701 images in COCO format from /workspace/data/labels_test_DT.json
[04/14 12:20:38 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[04/14 12:20:38 d2.data.common]: Serializing 2701 elements to byte tensors and concatenating them all ...
[04/14 12:20:38 d2.data.common]: Serialized dataset takes 1.22 MiB
[04/14 12:20:38 d2.evaluation.evaluator]: Start inference on 2701 batches
[04/14 12:20:39 d2.evaluation.evaluator]: Inference done 11/2701. Dataloading: 0.0008 s/iter. Inference: 0.0353 s/iter. Eval: 0.0002 s/iter. Total: 0.0